<font color = "blue"><p>
## Exercises

### Set Up

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

pd.set_option('max_columns', 15, 'max_rows', 20)

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [3]:
cs = pd.read_csv('Demo.csv', encoding='cp949')
tr = pd.read_csv('구매내역정보.csv', encoding='cp949', engine='python')

**[연습문제 28]** `60대 여성` 고객리스트를 출력하시오.

In [59]:
cs.query('성별 ==2 and 59 < 연령 < 70 ')

,ID,성별,연령,거주지역
222,55844757,2,69,서울 서대문구
223,165639634,2,69,부산 동래구
225,969901888,2,69,서울 동대문구
226,779190802,2,69,서울 용산구
228,414448990,2,69,경기 안산시
...,...,...,...,...
3516,473266991,2,69,서울 강서구
3524,330939633,2,60,경기 안양시
3532,524493815,2,61,부산 수영구
3539,168692065,2,63,경기 안양시


**[연습문제 29]** `남성고객`과 `여성고객`은 각각 몇명인가?

In [58]:
cs.groupby('성별').size()

성별
1     859
2    2683
dtype: int64

**[연습문제 30]** `여성고객`의 `평균나이`는 얼마인가?

In [61]:
cs.query('성별 == 2').연령.mean()

60.637718971300785

In [62]:
# 남녀별 평균나이
cs.groupby('성별')['연령'].mean().reset_index()

,성별,연령
0,1,62.442375
1,2,60.637719


**[연습문제 31]** `70대 여성 고객`들은 주로 어느 `지역`에 거주하고 있는가? 상위 5개 지역만 나열하시오.

In [67]:
cs.query('성별 == 2 and 69< 연령 <80').거주지역.value_counts()[:5]

경기 고양시     20
서울 송파구     13
경기 성남시     12
부산 해운대구    12
경기 안양시      9
Name: 거주지역, dtype: int64

**[연습문제 32]** (상품중분류명 기준) `Best seller`는 무엇인가?

In [68]:
tr.head(2)

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0


In [72]:
tr.상품중분류명.value_counts().index[0]

'가공식품'

**[연습문제 33]** `축산물`은 하루 중 언제 가장 많이 팔리는가?

In [74]:
tr.query('상품중분류명 =="축산물"').구매시간.value_counts().index[0]
#-1하면 제일 안팔리는 시간임

18

**[연습문제 34]** 전 지역에서 `판매량이 가장 많은 상품`의 `총 매출액`은 얼마인가?

In [75]:
x = tr.상품중분류명.value_counts().index[0]
tr.query('상품중분류명 == @x').구매금액.sum()
#@해줘야 밖의 변수를 query안에서 쓸 수 있다.

218379000

**[연습문제 35]** `60대 고객`의 `총 구매액`은 얼마인가?

In [76]:
pd.merge(cs,tr).query('60<= 연령 <=69').구매금액.sum()

2862716000

**[연습문제 36]** 각 `고객` 별로 `월` 별 `구매량`를 계산하여 출력하시오.

In [77]:
tr['월'] = tr.구매일자 // 100 % 100

In [79]:
pd.pivot_table(tr, index='ID', 
               columns = '월',
               values='구매수량', 
               aggfunc=np.sum, 
               fill_value=0).reset_index()

월,ID,1,2,3,4,5,6,7,8,9,10,11,12
0,741463,1,2,3,1,0,0,0,0,0,0,0,0
1,909249,0,1,0,4,3,2,0,6,0,0,0,2
2,1065068,0,0,0,0,0,0,2,0,0,5,0,4
3,1310410,2,0,0,0,0,0,0,0,0,1,0,0
4,1387032,1,2,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,998623524,0,4,1,0,4,3,0,0,3,0,1,5
3538,999024423,0,0,0,0,1,0,0,0,0,1,1,0
3539,999184958,3,0,2,0,7,3,0,7,0,2,0,0
3540,999375162,2,0,1,16,5,0,5,0,0,0,0,5


In [ ]:
#상품별 월별 몇개 팔리는지
pd.pivot_table(tr, index='상품대분류명', 
               columns = '월', 
               values='구매수량',
               aggfunc=np.sum, 
               fill_value=0).reset_index()

In [81]:
#성별 평균나이와 거주지역분포(서로다른 거주지역 수)를 집계하려한다.
cs.groupby('성별').agg({'연령':[('평균나이', np.mean)], '거주지역':[('거주지역 분포', Series.nunique)]})

,연령,거주지역
,평균나이,거주지역 분포
성별,,
1,62.442375,68
2,60.637719,71


In [4]:
#상품별로 가장 많이 팔리는 지역을 groupby을 이용하여 작성하시오
tr.groupby('상품중분류명')['구매지역'].agg(lambda x: x.value_counts().index[0])

상품중분류명
PC/노트북/프린터/카메라     경기 안양시
TV/AV             부산 해운대구
가공식품               서울 송파구
가구                  서울 중구
건강식품              부산 해운대구
                   ...   
침구류                경기 안양시
컴퓨터주변기기            서울 송파구
패션잡화               경기 안양시
화장품                서울 송파구
휴대폰/태블릿            서울 송파구
Name: 구매지역, Length: 31, dtype: object

In [83]:
#총구매액
total_buy = tr.groupby('ID')['구매금액'].sum().reset_index()
total_buy

,ID,구매금액
0,741463,560000
1,909249,640000
2,1065068,1103000
3,1310410,249000
4,1387032,113000
...,...,...
3537,998623524,2088000
3538,999024423,25000
3539,999184958,3147000
3540,999375162,10762000


In [84]:
cancel_buy = tr.query('취소여부 == 1').groupby('ID')['구매금액'].agg([('취소금액', np.sum)]).reset_index()
cancel_buy

,ID,취소금액
0,909249,-1149000
1,2272647,-1153000
2,2413953,-61000
3,2960045,-18000
4,3699906,-797000
...,...,...
1548,997853190,-2030000
1549,997894186,-2438000
1550,999184958,-124000
1551,999375162,-10754000


In [85]:
cancel_buy.shape, total_buy.shape

((1553, 2), (3542, 2))

In [86]:
pd.merge(total_buy, cancel_buy, how='left')

,ID,구매금액,취소금액
0,741463,560000,NaN
1,909249,640000,-1149000.0
2,1065068,1103000,NaN
3,1310410,249000,NaN
4,1387032,113000,NaN
...,...,...,...
3537,998623524,2088000,NaN
3538,999024423,25000,NaN
3539,999184958,3147000,-124000.0
3540,999375162,10762000,-10754000.0


**[연습문제 37]** `성별` 컬럼의 값을 `남`, `여`로 바꾸시오.

In [16]:
cs.성별.apply(lambda x: '남' if x == 1 else '여 ' )

0        남
1        남
2        남
3        남
4        남
        ..
3537     남
3538     남
3539    여 
3540    여 
3541    여 
Name: 성별, Length: 3542, dtype: object

In [17]:
cs.성별.map({1:'남', 2:'여'})

0       남
1       남
2       남
3       남
4       남
       ..
3537    남
3538    남
3539    여
3540    여
3541    여
Name: 성별, Length: 3542, dtype: object

**[연습문제 38]** `구매일자`에서 `요일`을 추출하여 새로운 컬럼 `판매요일`을 만드시오.

In [18]:
tr

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0
...,...,...,...,...,...,...,...,...,...
101687,797822538,가전제품,휴대폰/태블릿,서울 중구,20140528,22,1,0,0
101688,797822538,가전제품,휴대폰/태블릿,서울 중구,20140528,22,1,9000,0
101689,862978930,레포츠,스포츠용품/의류,서울 강서구,20140817,22,1,88000,0
101690,968743249,가전제품,TV/AV,경기 성남시,20141031,22,1,2569000,0


In [19]:
tr.구매일자.astype(str).astype('datetime64').dt.day_name().str.upper().str[:3] 
#숫자로 되어있으면 숫자->문자 ->datetime

0         SAT
1         THU
2         THU
3         THU
4         THU
         ... 
101687    WED
101688    WED
101689    SUN
101690    FRI
101691    FRI
Name: 구매일자, Length: 101692, dtype: object

In [20]:
tr["판매요일"] = tr.구매일자.astype(str).astype('datetime64').dt.day_name().str.upper().str[:3]

In [21]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부,판매요일
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0,SAT
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0,THU
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0,THU
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0,THU
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0,THU


**[연습문제 39]** `거주지역`을 시와 구 또는 도와 시로 나누어 두 개의 새로운 컬럼(`거주지역_광역`, `거주지역_기초`)을 만드시오.

In [22]:
cs.거주지역.apply(lambda x: x.split())
# ...

0        [서울, 성동구]
1        [서울, 서초구]
2        [부산, 사상구]
3       [서울, 영등포구]
4        [경기, 고양시]
           ...    
3537     [부산, 사하구]
3538     [경기, 수원시]
3539     [경기, 안양시]
3540     [경기, 의왕시]
3541     [경기, 안양시]
Name: 거주지역, Length: 3542, dtype: object

In [25]:
cs["거주지역_광역"] = cs.거주지역.apply(lambda x: x.split()[0])
cs["거주지역_기초"] = cs.거주지역.apply(lambda x: x.split()[1])

In [27]:
cs.head()

,ID,성별,연령,거주지역,거주지역_광역,거주지역_기초
0,478207946,1,84,서울 성동구,서울,성동구
1,479806984,1,84,서울 서초구,서울,서초구
2,94790213,1,84,부산 사상구,부산,사상구
3,656026338,1,84,서울 영등포구,서울,영등포구
4,433076833,1,83,경기 고양시,경기,고양시


**[연습문제 40]** 고객별로 `총구매건수`, `총구매액`, `마지막 구매시점부터 경과한 일수(2015년1월1일 현재)`을 계산하여 출력하시오. 

In [28]:
tr.head()

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부,판매요일
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,20140510,1,1,198000,0,SAT
1,643279402,식품,농산물,경기 성남시,20140612,1,1,2000,0,THU
2,643279402,식품,농산물,경기 성남시,20140612,1,1,4000,0,THU
3,643279402,식품,농산물,경기 성남시,20140612,1,1,5000,0,THU
4,643279402,식품,농산물,경기 성남시,20140612,1,1,9000,0,THU


In [29]:
#총구매건수
tr.groupby("ID").구매수량.sum().reset_index()

,ID,구매수량
0,741463,7
1,909249,18
2,1065068,11
3,1310410,3
4,1387032,6
...,...,...
3537,998623524,21
3538,999024423,3
3539,999184958,24
3540,999375162,34


In [30]:
#총구매액
tr.groupby("ID").구매금액.sum().reset_index()

,ID,구매금액
0,741463,560000
1,909249,640000
2,1065068,1103000
3,1310410,249000
4,1387032,113000
...,...,...
3537,998623524,2088000
3538,999024423,25000
3539,999184958,3147000
3540,999375162,10762000


In [32]:
tr["구매일자"] = tr["구매일자"].astype(str).astype('datetime64')
tr.head(2)

,ID,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부,판매요일
0,410362886,의류잡화,여성용의류-정장류,서울 송파구,2014-05-10,1,1,198000,0,SAT
1,643279402,식품,농산물,경기 성남시,2014-06-12,1,1,2000,0,THU


In [33]:
tr.groupby('ID')["구매일자"].agg(lambda x: (pd.to_datetime('2015-01-01')- x.max()).days)

ID
741463       275
909249         7
1065068        3
1310410       81
1387032       31
            ... 
998623524      3
999024423     42
999184958     77
999375162     11
999494396     47
Name: 구매일자, Length: 3542, dtype: int64

**[연습문제 41]** `서울`에 사는 고객 중 `총 구매액`이 가장 많은 사람부터 적은 사람 순으로 정렬하여 `상위 10명`만 출력하시오. 

In [36]:
# 나이 정보는 cs에 구매정보는 tr에 있기 때문에 먼저, 두 데이터프레임을 merge해야 한다.
md = pd.merge(cs, tr, on='ID')
md.head(1)

,ID,성별,연령,거주지역,거주지역_광역,거주지역_기초,상품대분류명,상품중분류명,구매지역,구매일자,구매시간,구매수량,구매금액,취소여부,판매요일
0,478207946,1,84,서울 성동구,서울,성동구,가전제품,컴퓨터주변기기,서울 동대문구,2014-12-19,13,1,59000,0,FRI


In [ ]:
# 총구매액이 가장 많은 사람부터 적은사람 순으로 정렬하여 상위 10명만 출력

In [37]:
md.groupby("ID").구매금액.sum().reset_index().sort_values(by='구매금액',ascending=False ).head(10)

,ID,구매금액
2426,700882892,136348000
256,67292818,58816000
546,152440226,50494000
2772,800536581,47498000
2802,808654096,43491000
2454,709252477,39084000
3486,988927928,38500000
2303,658972837,35151000
1008,287927197,34642000
470,126094176,31951000
